In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [62]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, extract, inspect 
from sqlalchemy import * 
import sqlalchemy as sa 

metadata = MetaData()

In [51]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [41]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [42]:
# We can view all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [43]:
# Save references to each table
measurement = Base.classes.measurement

station = Base.classes.station

In [44]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [45]:
#Viewing the data from the reference 
engine.execute('SELECT * FROM measurement LIMIT (10) ').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', None, 73.0),
 (6, 'USC00519397', '2010-01-07', 0.06, 70.0),
 (7, 'USC00519397', '2010-01-08', 0.0, 64.0),
 (8, 'USC00519397', '2010-01-09', 0.0, 68.0),
 (9, 'USC00519397', '2010-01-10', 0.0, 73.0),
 (10, 'USC00519397', '2010-01-11', 0.01, 64.0)]

In [46]:
engine.execute('SELECT * FROM station LIMIT (11)').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [64]:
#Using inspector to explore the database
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [67]:
#obtaining column names from measurement table
columns = inspector.get_columns('measurement')

for c in columns: 
    print(c["name"],c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [68]:
#obtaining column names from measurement table
columns = inspector.get_columns('station')

for c in columns: 
    print(c["name"],c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [69]:
#grabbing a count of total days 
session.query(func.count(measurement.date)).all()

[(19550)]

In [70]:
#printing the earliest date 
session.query(measurement.date).order_by(measurement.date).first()

('2010-01-01')

In [72]:
#printing the last date 
session.query(measurement.date).order_by(measurement.date.desc()).first()

('2017-08-23')

In [78]:
# Calculate the date 1 year ago from the last data point in the database
query_date = dt.date(2017,8,23) - dt.timedelta(days=365)

print(f"Date 1 year from the last date poitn: ",query_date)

Date 1 year from the last date poitn:  2016-08-23


In [86]:
# Perform a query to retrieve the data and precipitation scores
prcp_query = session.query(measurement.id,
                          measurement.station,
                          measurement.date,
                          measurement.prcp,
                          measurement.tobs).\
            filter(measurement.date >= query_date).\
            order_by(measurement.date).all()

prcp_query

[(2364, 'USC00519397', '2016-08-23', 0.0, 81.0),
 (5091, 'USC00513117', '2016-08-23', 0.15, 76.0),
 (7331, 'USC00514830', '2016-08-23', 0.05, 80.0),
 (8825, 'USC00517948', '2016-08-23', None, 80.0),
 (11851, 'USC00519523', '2016-08-23', 0.02, 80.0),
 (14608, 'USC00519281', '2016-08-23', 1.79, 77.0),
 (19202, 'USC00516128', '2016-08-23', 0.7, 74.0),
 (2365, 'USC00519397', '2016-08-24', 0.08, 79.0),
 (5092, 'USC00513117', '2016-08-24', 2.15, 76.0),
 (7332, 'USC00514830', '2016-08-24', 2.28, 80.0),
 (8826, 'USC00517948', '2016-08-24', None, 78.0),
 (11852, 'USC00519523', '2016-08-24', 1.22, 79.0),
 (14609, 'USC00519281', '2016-08-24', 2.15, 77.0),
 (19203, 'USC00516128', '2016-08-24', 1.45, 74.0),
 (2366, 'USC00519397', '2016-08-25', 0.08, 80.0),
 (5093, 'USC00513117', '2016-08-25', 0.08, 77.0),
 (7333, 'USC00514830', '2016-08-25', 0.0, 81.0),
 (8827, 'USC00517948', '2016-08-25', 0.0, 81.0),
 (11853, 'USC00519523', '2016-08-25', 0.21, 80.0),
 (14610, 'USC00519281', '2016-08-25', 0.06, 80.

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
#Grouping dates by year to narrow down my results then retrieve the last 12 months of percipitation 
group_dates = session.query(measurement.date).order_by(measurement.date.desc()).first()
print(group_dates)

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
past_year = dt.date(2017,8,23) - dt.timedelta(days=365)
print(past_year)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
